# Preface
- この Notebook は、[Pandas: Column/Series  - dt accessor で日付/時刻を一発で取り出したい](https://ggcs.io/2020/10/26/pandas-accessors-dt) の手順詳細版です。¶
  - Website: ごたごた気流調査所 https://ggcs.io
  - GitHub : Gota Gota Current Survey https://github.com/ggcurrs/gota2-observatory
  - Version 1.1.0
  - Date : 2020-10-25
  - Updated: 2020-10-26

# Data 読み込み
- CSVなどから元データ（テキストデータ）を Pandas DataFreme （以下、「DF」）に読み込みます。

In [9]:
import pandas as pd

df = pd.DataFrame(
    {'DATE_AND_TIME':
         ['2020-01-01 06:50',
          '2020-01-01 10:50',
          '2020-01-01 16:38',
          '2020-01-01 22:16'],
     'EVENT':
         ['Sunrise in Tokyo',
          'Moonrise in Tokyo',
          'Sunset in Tokyo',
          'Moonset in Tokyo']
     }
)

# This DF is spared for later use...🤫
df2 = df.copy()

# Confirm
df.head()

,DATE_AND_TIME,EVENT
0,2020-01-01 06:50,Sunrise in Tokyo
1,2020-01-01 10:50,Moonrise in Tokyo
2,2020-01-01 16:38,Sunset in Tokyo
3,2020-01-01 22:16,Moonset in Tokyo


# 日時データを Timestamp に変換
- DATE_AND_TIME 列（Series）を取り出して見ると、読み込んだばかりなのでデータはもちろんただの str （テキストデータ）で、Pandas としては日付・時刻データだとは思っていませんから、何かと不便です。

In [10]:
# Raw DataFreme
df.DATE_AND_TIME

0    2020-01-01 06:50
1    2020-01-01 10:50
2    2020-01-01 16:38
3    2020-01-01 22:16
Name: DATE_AND_TIME, dtype: object

## pandas.to_datetime() 登場！
- そこで、秘技「pandas.to_datatime()」を使って data type を datetime に変換します。

cf. [Pandas: CSVに文字型で記録されている日付と時間をPandasに読み込んだ後、日付・時刻型に一発変換したい](https://ggcs.io/2020/05/10/pandas-date-time-format/)

In [11]:
# Convert object(str) to datetime 
df.DATE_AND_TIME = pd.to_datetime(df.DATE_AND_TIME)

# Confirm
df.DATE_AND_TIME

0   2020-01-01 06:50:00
1   2020-01-01 10:50:00
2   2020-01-01 16:38:00
3   2020-01-01 22:16:00
Name: DATE_AND_TIME, dtype: datetime64[ns]

## 日付データだけ抽出したい

- この表のデータは全部 2020 年の元日のものなので、日付は冗長でいらないかも、ということで、時刻データだけを取り出すことにします。
- 元の str データのままだったら slice で取り出したり、場合によっては regex で切り出したりといろいろ面倒臭い話になるところでしたが、たったいま datetime 形式に変換したので、日付だろうが時刻だろうが自由自在に取り出せます。
- ということで、`.strftime()` を使って時刻データを取り出すことにします（取り出し後は str データになります）。

### strftime で日付文字列（str）に変換するぞ！　➡️　あれれ？！

In [12]:
# strftime を使うためにはこれ👇が必要です。
from datetime import datetime

df.DATE_AND_TIME.strftime('%H:%M:%S')

AttributeError: ignored

- `Series' object has no attribute 'strftime'` （Series に strftime アトリビュートは無いよ）とかどうとか言ってる。Excel みたいに列を一発変換とかできないの？不便だな…。`SEARCH STACK OVERFLOW` リンクを辿るのも面倒だからここはササッと loop で回しちゃった方が早いだろう。
- ほら、簡単にできちゃた！天才だな👺。

In [13]:
# 'f'ormat timedate to str.
for i in range(len(df)):
  df.at[i, 'DATE_AND_TIME'] = df.at[i, 'DATE_AND_TIME'].strftime('%H:%M:%S')

# Confirm
df
# できた、できた！

,DATE_AND_TIME,EVENT
0,06:50:00,Sunrise in Tokyo
1,10:50:00,Moonrise in Tokyo
2,16:38:00,Sunset in Tokyo
3,22:16:00,Moonset in Tokyo


# あんなぁ… 
- などと蘊蓄を垂れていたら いずこからともなく神様の声が…
「あんなぁ、Colab はんが親切に `SEARCH STACK OVERFLOW` のリンクを貼ってくださってるんやから素直に読んだらええやんか？Pandas の三種の神器 dt, str, cat も知らずに loop とか抜かしてる場合やないで、ホンマ💢」
- 「`pd.to_datetime` のあと、`dt` アクセサを使えば 1行で済む話！」

えらいすんまへん 🙇‍♂️

## ということで、仕切り直し。DF はやり直し用に用意しておいた df2 を使います。

In [14]:
# Startng over.
# Convert object(str) to datetime 
df2.DATE_AND_TIME = pd.to_datetime(df2.DATE_AND_TIME)
# Confirm
df2.DATE_AND_TIME

0   2020-01-01 06:50:00
1   2020-01-01 10:50:00
2   2020-01-01 16:38:00
3   2020-01-01 22:16:00
Name: DATE_AND_TIME, dtype: datetime64[ns]

## で、`dt` （たぶん datetime の略）アクセサを使えば 1 行でおしまい😭

In [15]:
df2.DATE_AND_TIME = df2.DATE_AND_TIME.dt.time

# Confirm
df2.DATE_AND_TIME

# Note that the dtype is 'object' (str).

0    06:50:00
1    10:50:00
2    16:38:00
3    22:16:00
Name: DATE_AND_TIME, dtype: object

- 折角なので、列名（column name）を rename しておきます。

In [16]:
# Rename the column DATE_AND_TIME to TIME
df2.rename(columns={'DATE_AND_TIME': 'TIME'}, inplace=True)
df2

,TIME,EVENT
0,06:50:00,Sunrise in Tokyo
1,10:50:00,Moonrise in Tokyo
2,16:38:00,Sunset in Tokyo
3,22:16:00,Moonset in Tokyo


**Enjoy!**